In [1]:
# module import
import os
import sys
from datetime import datetime
import pickle

import numpy as np
import PIL.Image
import torch
import torchvision
                
sys.path.append('../cnn_preferred')
from utils import normalise_img, clip_extreme_pixel,  get_cnn_features, img_deprocess, get_target_feature_shape
from activation_maximization import generate_preferred

In [2]:
## load network
net = torchvision.models.vgg16(pretrained=True)
net.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [3]:
# image mean and std for pre/de-process image for input network
img_mean=np.array([0.485, 0.456, 0.406],dtype=np.float),
img_std = np.array([0.229,0.224,0.225])

# if the model input is for 0-1 range, norm = 255, elif 0-255, norm = 1
norm = 255

In [4]:
# create save_dir
save_dir = '../result'
save_folder = 'jupyter_demo_torch_simpleCNN_lower'#__file__.split('.')[0]
save_folder = save_folder + '_' + datetime.now().strftime('%Y%m%dT%H%M%S')
save_path = os.path.join(save_dir,save_folder)
os.makedirs(save_path, exist_ok=True)

In [5]:
# initial image for the optimization

h, w = 224,224
initial_image = np.zeros(( h,w,3),dtype='float32')


initial_input = np.random.randint(0, 256, (h,w,3))

In [6]:
net

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [19]:

target_layer = "net.features[8]"
target_layer = "classifier[3]"
extract_list = [target_layer]

In [14]:
#targetlayer
exec_str_list = [target_layer]


exec("num_of_ch = net."+target_layer+".weight.detach().numpy().shape[0]") #param_list[layer_value].shape[0]
num_of_img = 10
step = int(num_of_ch/num_of_img)
channel_list = range(0,num_of_ch,step)

ee = get_cnn_features(net,torch.Tensor(initial_input.transpose(2,0, 1)[np.newaxis]), exec_str_list)
feat_num = ee[0].detach().numpy().shape

In [15]:
# options
opts = {
    'img_mean': img_mean, # img_mean to preprocessing input image (the default is [0.485, 0.456, 0.406]) 
    'img_std': img_std,   # img_std to preprocessing input image  (the default is [0.229,0.224,0.225]) 
    'norm': norm,         # if the model input is for 0-1 range, norm = 255, elif 0-255, norm = 1 (defalt is 255)
    'bgr': False,         # if True, input color is adjust to BGR input from RGB input. (default False)
    
    'iter_n': 200, # the total number of iterations for gradient descend (defalt is 200)

    'disp_every': 1, # display the information on the terminal for every n iterations (default is 1)

    'save_intermediate': True, # save the intermediate or not (default is None)
    'save_intermediate_every': 10, # save the intermediate for every n iterations (default is 10)
    'save_intermediate_path': save_path, # the path to save the intermediate (default is None)

    'lr_start': 1., # learning rate (default is 1.)
    'lr_end': 1.,   # we can change learning rate linearly setteing these two parameters 

    'momentum_start': 0.001, # gradient with momentum (default is 0.001)
    'momentum_end': 0.001,   # we can change momentum linearly setteing these two parameters too 

    'decay_start': 0.001, # pixel decay for each iteration (default is 0.001)
    'decay_end': 0.001,   # we can also change pixel decay linealy  

    'image_blur': True, # Use image smoothing or not (default is True)
    'sigma_start': 2.5, # the size of the gaussian filter for image smoothing (default is 2.5)
    'sigma_end': 0.5,   

    'image_jitter': True, # use image jittering during optimization (default is True)
    'jitter_size': 4,     # the size of jitter (default is 32)
    
    'use_p_norm_reg': False, # use p_norm regularization (default is False)
    'p': 2,

    'use_TV_norm_reg': False, # use total variance norm (default is False)
    'TVbeta1': 1,             # the order of  spatial domain
    'TVbeta2':1.2,            # the order temporal domain (for video input)

    'clip_small_norm': True,   # clip or not the pixels with extreme high or low value (default True) 
    'clip_small_norm_every': 1,
    'n_pct_start': 5,
    'n_pct_end': 5,

    'clip_small_contribution': True, # clip or not the poxels with smal contribution norm of RGB channels (default True)
    'clip_small_contribution_every': 1,
    'c_pct_start': 5,
    'c_pct_end':5,
    
    'initial_input': initial_input, # the initial image for the optimization (setting to None will use random noise as initial image)
    }


In [16]:
channel_list = np.sort(np.random.choice(np.arange(feat_num[1]), 4))

In [17]:
feat_num

(1, 4096)

In [ ]:
for channel in channel_list:
    #
    print('')
    print('channel='+str(channel))
    print('')
    # activation maximization
    preferred_stim = generate_preferred(net, exec_str_list, feature_mask=feature_mask, **opts)
    
    # if you want to preferred image for unit, not chanel, you can additionaly define feature mask, which is 
    # binaly mask that target unit is 1 the other is 0.
    
    ## target units
    #feat_size = feat_shape
    #y_index = int(feat_size[2]/2) # the unit in the center of feature map
    #x_index = int(feat_size[3]/2) # the unit in the center of feature map
    #feature_mask = np.zeros(feat_size)
    #feature_mask[0,channel,y_index,x_index] = 1
    ## activation maximization
    #preferred_stim = generate_preferred(net, exec_str_list, feature_mask=feature_mask, **opts)
    
    
    # save the results
    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.npy'
    np.save(os.path.join(save_path,save_name), preferred_stim)

    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.jpg'
    # To better display the image, clip pixels with extreme values (0.02% of
    # pixels with extreme low values and 0.02% of the pixels with extreme high
    # values). And then normalise the image by mapping the pixel value to be
    # within [0,255].
    PIL.Image.fromarray(normalise_img(clip_extreme_pixel(preferred_stim, pct=0.04))).save(
                    os.path.join(save_path, save_name))


channel=162

iter=1; mean(abs(feat))=0.807117;
iter=2; mean(abs(feat))=1.17887;
iter=3; mean(abs(feat))=0.320728;
iter=4; mean(abs(feat))=0.730028;
iter=5; mean(abs(feat))=1.23964;
iter=6; mean(abs(feat))=1.12779;
iter=7; mean(abs(feat))=1.55625;
iter=8; mean(abs(feat))=1.22882;
iter=9; mean(abs(feat))=1.34386;
iter=10; mean(abs(feat))=2.00903;
iter=11; mean(abs(feat))=1.90427;
iter=12; mean(abs(feat))=3.21309;
iter=13; mean(abs(feat))=3.13195;
iter=14; mean(abs(feat))=1.89007;
iter=15; mean(abs(feat))=2.97331;
iter=16; mean(abs(feat))=3.21518;
iter=17; mean(abs(feat))=4.2499;
iter=18; mean(abs(feat))=4.1205;
iter=19; mean(abs(feat))=5.07067;
iter=20; mean(abs(feat))=5.32524;
iter=21; mean(abs(feat))=6.56858;
iter=22; mean(abs(feat))=5.79554;
iter=23; mean(abs(feat))=6.17014;
iter=24; mean(abs(feat))=5.66237;
iter=25; mean(abs(feat))=7.70532;
iter=26; mean(abs(feat))=5.10501;
iter=27; mean(abs(feat))=4.95845;
iter=28; mean(abs(feat))=5.45;
iter=29; mean(abs(feat))=6.18898;
iter=30; me

iter=239; mean(abs(feat))=14.7181;
iter=240; mean(abs(feat))=15.348;
iter=241; mean(abs(feat))=15.2333;
iter=242; mean(abs(feat))=16.8074;
iter=243; mean(abs(feat))=13.8516;
iter=244; mean(abs(feat))=14.5418;
iter=245; mean(abs(feat))=15.4163;
iter=246; mean(abs(feat))=14.4251;
iter=247; mean(abs(feat))=16.4597;
iter=248; mean(abs(feat))=14.4691;
iter=249; mean(abs(feat))=12.4546;
iter=250; mean(abs(feat))=15.9101;
iter=251; mean(abs(feat))=15.8243;
iter=252; mean(abs(feat))=15.0758;
iter=253; mean(abs(feat))=17.075;
iter=254; mean(abs(feat))=13.1798;
iter=255; mean(abs(feat))=15.7787;
iter=256; mean(abs(feat))=12.2197;
iter=257; mean(abs(feat))=13.5993;
iter=258; mean(abs(feat))=13.2585;
iter=259; mean(abs(feat))=12.013;
iter=260; mean(abs(feat))=15.497;
iter=261; mean(abs(feat))=15.5271;
iter=262; mean(abs(feat))=15.0782;
iter=263; mean(abs(feat))=17.722;
iter=264; mean(abs(feat))=13.9724;
iter=265; mean(abs(feat))=14.9745;
iter=266; mean(abs(feat))=14.431;
iter=267; mean(abs(feat))=

iter=474; mean(abs(feat))=17.2231;
iter=475; mean(abs(feat))=18.2686;
iter=476; mean(abs(feat))=18.8458;
iter=477; mean(abs(feat))=19.6038;
iter=478; mean(abs(feat))=21.3805;
iter=479; mean(abs(feat))=17.5803;
iter=480; mean(abs(feat))=19.1355;
iter=481; mean(abs(feat))=18.0194;
iter=482; mean(abs(feat))=18.2079;
iter=483; mean(abs(feat))=12.8726;
iter=484; mean(abs(feat))=17.1551;
iter=485; mean(abs(feat))=16.0504;
iter=486; mean(abs(feat))=18.2819;
iter=487; mean(abs(feat))=17.9514;
iter=488; mean(abs(feat))=18.5342;
iter=489; mean(abs(feat))=16.6706;
iter=490; mean(abs(feat))=15.7736;
iter=491; mean(abs(feat))=17.6465;
iter=492; mean(abs(feat))=18.7505;
iter=493; mean(abs(feat))=19.6471;
iter=494; mean(abs(feat))=23.731;
iter=495; mean(abs(feat))=21.95;
iter=496; mean(abs(feat))=19.1528;
iter=497; mean(abs(feat))=17.332;
iter=498; mean(abs(feat))=21.698;
iter=499; mean(abs(feat))=22.3263;
iter=500; mean(abs(feat))=22.065;
iter=501; mean(abs(feat))=21.0026;
iter=502; mean(abs(feat))=

iter=709; mean(abs(feat))=23.9813;
iter=710; mean(abs(feat))=27.8038;
iter=711; mean(abs(feat))=25.4681;
iter=712; mean(abs(feat))=26.3648;
iter=713; mean(abs(feat))=27.0203;
iter=714; mean(abs(feat))=25.2768;
iter=715; mean(abs(feat))=26.5203;
iter=716; mean(abs(feat))=22.583;
iter=717; mean(abs(feat))=23.6006;
iter=718; mean(abs(feat))=26.459;
iter=719; mean(abs(feat))=26.7701;
iter=720; mean(abs(feat))=23.086;
iter=721; mean(abs(feat))=21.6395;
iter=722; mean(abs(feat))=24.0573;
iter=723; mean(abs(feat))=23.6309;
iter=724; mean(abs(feat))=23.7235;
iter=725; mean(abs(feat))=26.5578;
iter=726; mean(abs(feat))=25.3343;
iter=727; mean(abs(feat))=26.2967;
iter=728; mean(abs(feat))=28.1384;
iter=729; mean(abs(feat))=26.7051;
iter=730; mean(abs(feat))=23.7434;
iter=731; mean(abs(feat))=28.8476;
iter=732; mean(abs(feat))=24.2548;
iter=733; mean(abs(feat))=25.1754;
iter=734; mean(abs(feat))=23.7953;
iter=735; mean(abs(feat))=24.5777;
iter=736; mean(abs(feat))=22.603;
iter=737; mean(abs(feat)

iter=944; mean(abs(feat))=35.4094;
iter=945; mean(abs(feat))=40.2149;
iter=946; mean(abs(feat))=36.381;
iter=947; mean(abs(feat))=37.009;
iter=948; mean(abs(feat))=41.0609;
iter=949; mean(abs(feat))=35.5975;
iter=950; mean(abs(feat))=39.7671;
iter=951; mean(abs(feat))=35.7923;
iter=952; mean(abs(feat))=38.3976;
iter=953; mean(abs(feat))=41.3166;
iter=954; mean(abs(feat))=37.6388;
iter=955; mean(abs(feat))=43.6187;
iter=956; mean(abs(feat))=32.9367;
iter=957; mean(abs(feat))=38.059;
iter=958; mean(abs(feat))=41.2901;
iter=959; mean(abs(feat))=43.2621;
iter=960; mean(abs(feat))=41.7115;
iter=961; mean(abs(feat))=43.0135;
iter=962; mean(abs(feat))=39.4122;
iter=963; mean(abs(feat))=39.8721;
iter=964; mean(abs(feat))=34.3693;
iter=965; mean(abs(feat))=33.7207;
iter=966; mean(abs(feat))=36.6683;
iter=967; mean(abs(feat))=34.3606;
iter=968; mean(abs(feat))=37.627;
iter=969; mean(abs(feat))=38.2753;
iter=970; mean(abs(feat))=37.1444;
iter=971; mean(abs(feat))=40.7799;
iter=972; mean(abs(feat)

...

iter=1720; mean(abs(feat))=343.141;
iter=1721; mean(abs(feat))=355.02;
iter=1722; mean(abs(feat))=325.42;
iter=1723; mean(abs(feat))=372.684;
iter=1724; mean(abs(feat))=343.706;
iter=1725; mean(abs(feat))=356.29;
iter=1726; mean(abs(feat))=351.513;
iter=1727; mean(abs(feat))=358.461;
iter=1728; mean(abs(feat))=381.062;
iter=1729; mean(abs(feat))=378.331;
iter=1730; mean(abs(feat))=322.276;
iter=1731; mean(abs(feat))=324.847;
iter=1732; mean(abs(feat))=369.143;
iter=1733; mean(abs(feat))=343.223;
iter=1734; mean(abs(feat))=345.63;
iter=1735; mean(abs(feat))=363.544;
iter=1736; mean(abs(feat))=387.1;
iter=1737; mean(abs(feat))=381.477;
iter=1738; mean(abs(feat))=319.148;
iter=1739; mean(abs(feat))=369.416;
iter=1740; mean(abs(feat))=348.747;
iter=1741; mean(abs(feat))=358.044;
iter=1742; mean(abs(feat))=390.938;
iter=1743; mean(abs(feat))=336.486;
iter=1744; mean(abs(feat))=370.003;
iter=1745; mean(abs(feat))=342.622;
iter=1746; mean(abs(feat))=355.168;
iter=1747; mean(abs(feat))=366.972

iter=1949; mean(abs(feat))=785.393;
iter=1950; mean(abs(feat))=780.559;
iter=1951; mean(abs(feat))=829.872;
iter=1952; mean(abs(feat))=753.856;
iter=1953; mean(abs(feat))=805.552;
iter=1954; mean(abs(feat))=824.022;
iter=1955; mean(abs(feat))=664.167;
iter=1956; mean(abs(feat))=880.33;
iter=1957; mean(abs(feat))=913.327;
iter=1958; mean(abs(feat))=824.793;
iter=1959; mean(abs(feat))=826.519;
iter=1960; mean(abs(feat))=787.397;
iter=1961; mean(abs(feat))=828.033;
iter=1962; mean(abs(feat))=900.33;
iter=1963; mean(abs(feat))=896.307;
iter=1964; mean(abs(feat))=877.176;
iter=1965; mean(abs(feat))=997.023;
iter=1966; mean(abs(feat))=831.811;
iter=1967; mean(abs(feat))=887.766;
iter=1968; mean(abs(feat))=889.422;
iter=1969; mean(abs(feat))=928.099;
iter=1970; mean(abs(feat))=910.959;
iter=1971; mean(abs(feat))=897.116;
iter=1972; mean(abs(feat))=897.936;
iter=1973; mean(abs(feat))=857.919;
iter=1974; mean(abs(feat))=929.004;
iter=1975; mean(abs(feat))=924.812;
iter=1976; mean(abs(feat))=965

iter=185; mean(abs(feat))=3.10808;
iter=186; mean(abs(feat))=4.33116;
iter=187; mean(abs(feat))=3.50629;
iter=188; mean(abs(feat))=2.94695;
iter=189; mean(abs(feat))=4.0984;
iter=190; mean(abs(feat))=3.15821;
iter=191; mean(abs(feat))=2.54094;
iter=192; mean(abs(feat))=4.10005;
iter=193; mean(abs(feat))=5.49606;
iter=194; mean(abs(feat))=2.559;
iter=195; mean(abs(feat))=3.74931;
iter=196; mean(abs(feat))=3.40058;
iter=197; mean(abs(feat))=4.54604;
iter=198; mean(abs(feat))=4.1287;
iter=199; mean(abs(feat))=3.12;
iter=200; mean(abs(feat))=3.62041;
iter=201; mean(abs(feat))=5.2135;
iter=202; mean(abs(feat))=3.20287;
iter=203; mean(abs(feat))=3.98039;
iter=204; mean(abs(feat))=3.52548;
iter=205; mean(abs(feat))=3.95928;
iter=206; mean(abs(feat))=4.672;
iter=207; mean(abs(feat))=4.27311;
iter=208; mean(abs(feat))=3.19225;
iter=209; mean(abs(feat))=4.31865;
iter=210; mean(abs(feat))=3.77909;
iter=211; mean(abs(feat))=5.11506;
iter=212; mean(abs(feat))=3.97115;
iter=213; mean(abs(feat))=3.79

iter=421; mean(abs(feat))=4.41403;
iter=422; mean(abs(feat))=4.24474;
iter=423; mean(abs(feat))=4.13218;
iter=424; mean(abs(feat))=4.79888;
iter=425; mean(abs(feat))=6.84401;
iter=426; mean(abs(feat))=5.06779;
iter=427; mean(abs(feat))=4.2798;
iter=428; mean(abs(feat))=4.8775;
iter=429; mean(abs(feat))=3.7251;
iter=430; mean(abs(feat))=4.92488;
iter=431; mean(abs(feat))=5.76001;
iter=432; mean(abs(feat))=4.90829;
iter=433; mean(abs(feat))=4.86062;
iter=434; mean(abs(feat))=4.71608;
iter=435; mean(abs(feat))=5.52049;
iter=436; mean(abs(feat))=4.34831;
iter=437; mean(abs(feat))=3.51893;
iter=438; mean(abs(feat))=3.95052;
iter=439; mean(abs(feat))=3.22475;
iter=440; mean(abs(feat))=4.43041;
iter=441; mean(abs(feat))=4.89105;
iter=442; mean(abs(feat))=4.02194;
iter=443; mean(abs(feat))=3.25659;
iter=444; mean(abs(feat))=5.07296;
iter=445; mean(abs(feat))=3.36958;
iter=446; mean(abs(feat))=4.42736;
iter=447; mean(abs(feat))=4.85828;
iter=448; mean(abs(feat))=3.43061;
iter=449; mean(abs(feat

iter=656; mean(abs(feat))=7.74644;
iter=657; mean(abs(feat))=5.50968;
iter=658; mean(abs(feat))=5.9507;
iter=659; mean(abs(feat))=6.47744;
iter=660; mean(abs(feat))=6.31999;
iter=661; mean(abs(feat))=6.15457;
iter=662; mean(abs(feat))=5.41547;
iter=663; mean(abs(feat))=5.23851;
iter=664; mean(abs(feat))=3.7339;
iter=665; mean(abs(feat))=6.73322;
iter=666; mean(abs(feat))=6.45475;
iter=667; mean(abs(feat))=5.94445;
iter=668; mean(abs(feat))=7.0189;
iter=669; mean(abs(feat))=7.70127;
iter=670; mean(abs(feat))=3.97772;
iter=671; mean(abs(feat))=6.42741;
iter=672; mean(abs(feat))=6.43897;
iter=673; mean(abs(feat))=6.57545;
iter=674; mean(abs(feat))=6.00494;
iter=675; mean(abs(feat))=6.45666;
iter=676; mean(abs(feat))=7.39871;
iter=677; mean(abs(feat))=7.74136;
iter=678; mean(abs(feat))=7.97146;
iter=679; mean(abs(feat))=9.17694;
iter=680; mean(abs(feat))=8.30995;
iter=681; mean(abs(feat))=8.82399;
iter=682; mean(abs(feat))=6.89363;
iter=683; mean(abs(feat))=6.66769;
iter=684; mean(abs(feat